<h1><strong><font color='green'>Feature Engineering</font></strong></h1>

As part of Feature engineering we will extract these features from the text data :
<li>Creating tfidf-w2v representation using pretrained glove model (300-dim)</li>
<li>Length of the text by word level and character level (2-dim)</li>
<li>Sentiment scores of text data using nltk (4-dim)</li>

We will get a total of 306 features.

In [ ]:
#Importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
import pickle
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy

In [ ]:
#Creating input and target data

X_train,y_train=pd.DataFrame(final_train.iloc[:,0]),final_train.iloc[:,1:]
X_cv,y_cv=pd.DataFrame(final_cv.iloc[:,0]),final_cv.iloc[:,1:]
X_test,y_test=pd.DataFrame(final_test.iloc[:,0]),final_test.iloc[:,1:]

In [ ]:
print('Shapes of train,cross validate and test data :')
print('X_ train shape : ',X_train.shape,',  y_train shape : ', y_train.shape)
print('X_ cv shape :     ',X_cv.shape, '   ,  y_cv shape : ',y_cv.shape)
print('X_ test shape :  ',X_test.shape, ',  y_test shape : ',y_test.shape)

Shapes of train,cross validate and test data :
X_ train shape :  (6659, 1) ,  y_train shape :  (6659, 3)
X_ cv shape :      (940, 1)    ,  y_cv shape :  (940, 3)
X_ test shape :   (1608, 1) ,  y_test shape :  (1608, 3)


In [ ]:
fasttext_word_embeddings = dict()
fasttext_model = open('wiki-news-300d-1M-subword.vec',encoding='utf-8')
for line in fasttext_model: # For each line in vector file
  row = line.split() 
  word = row[0] # First word 
  vector = np.asarray(row[1:], dtype='float32') # Remaining text is 300 dimensional embeddings
  fasttext_word_embeddings[word] = vector # Creating a dictionay with word as key and embeddings  vector as value
fasttext_model.close()

fasttext_words=fasttext_word_embeddings.keys()

In [ ]:
words_to_emebd=set()
for index,row in X_train.iterrows():
  text=row['Description'].split()
  for word in text:
    words_to_emebd.add(word)

In [ ]:
for index,row in X_cv.iterrows():
  text=row['Description'].split()
  for word in text:
    words_to_emebd.add(word)

In [ ]:
for index,row in X_test.iterrows():
  text=row['Description'].split()
  for word in text:
    words_to_emebd.add(word)

In [ ]:
#Getting the word embedding from pretrained glove model.

glove_word_embeddings = dict()
glove_model = open('glove.6B.300d.txt')
for line in glove_model: # For each line in text file
  row = line.split() 
  word = row[0] # First word 
  if word in words_to_embed:
    vector = np.asarray(row[1:], dtype='float32') # Remaining text is 300 dimensional embeddings
    glove_word_embeddings[word] = vector # Creating a dictionay with word as key and embeddings  vector as value
glove_model.close()

glove_words=glove_word_embeddings.keys()

In [ ]:
pickle.dump(glove_word_embeddings,open('small_glove_embeddings.pkl','wb'))

In [ ]:
def get_embedding_features(data,word_embeddings,model_words):
  '''This function takes dataframe as input and returns fasttext vecotr respresent of text data(Description)'''
  vector_rep=[]
  preprocessed_descriptions = data['Description'].values
  for text in preprocessed_descriptions: # For each description
    vector=np.zeros(300)
    n=0
    for word in text.split():# For each word in vector
      if (word in model_words):
        vec=word_embeddings[word] #Getting the word's w2v representation
        vector+=vec
        n+=1
    if n!=0:
      vector/=n
    vector_rep.append(vector)
  return np.array(vector_rep)

In [ ]:
tfidf_model = TfidfVectorizer() 
tfidf_model.fit(X_train['Description'].values) # Fitting the tfidf vectorizer on train data
# we are converting a dictionary with word as a key, and the idf as a value
idf_dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())

In [ ]:
def tfidf_w2v(data,word_embeddings,model_words):
  '''This function takes dataframe as input and returns tfidf-w2v vecotr respresent of text data(Description)'''
  tfidf_w2v_vectors=[]
  preprocessed_descriptions = data['Description'].values
  for text in preprocessed_descriptions: # For each description
    vector=np.zeros(300) #Intialising a 300-dim vector
    tf_idf_weight =0
    for word in text.split(): # For each word in vector
      if (word in model_words) and (word in tfidf_words):
        vec=word_embeddings[word] #Getting the word's w2v representation
        tf_idf = idf_dictionary[word]*(text.count(word)/len(text.split())) # Calulating tfidf value of word using idf values
        vector += (vec * tf_idf) # Computing weighted sum of tfidf-w2v
        tf_idf_weight += tf_idf
    if tf_idf_weight!=0:
      vector/=tf_idf_weight # Averaging the weighted sum of tfidf-w2v
    tfidf_w2v_vectors.append(vector)
  return np.array(tfidf_w2v_vectors)

In [ ]:
X_train_fasttext_w2v=get_embedding_features(X_train,fasttext_word_embeddings,fasttext_words)
X_cv_fasttext_w2v=get_embedding_features(X_cv,fasttext_word_embeddings,fasttext_words)
X_test_fasttext_w2v=get_embedding_features(X_test,fasttext_word_embeddings,fasttext_words)
X_train_fasttext_w2v=pd.DataFrame(X_train_fasttext_w2v,columns=['embed_'+str(i) for i in range(300)])
X_cv_fasttext_w2v=pd.DataFrame(X_cv_fasttext_w2v,columns=['embed_'+str(i) for i in range(300)])
X_test_fasttext_w2v=pd.DataFrame(X_test_fasttext_w2v,columns=['embed_'+str(i) for i in range(300)])
print('\nAfter fasttext-w2v\n')
print('Shapes of tfidf-w2v features on train data : ',X_train_fasttext_w2v.shape)
print('Shapes of tfidf-w2v features on cv data : ',X_cv_fasttext_w2v.shape)
print('Shapes of tfidf-w2v features on test data : ',X_test_fasttext_w2v.shape)


After fasttext-w2v

Shapes of tfidf-w2v features on train data :  (6659, 300)
Shapes of tfidf-w2v features on cv data :  (940, 300)
Shapes of tfidf-w2v features on test data :  (1608, 300)


In [ ]:
X_train_fasttext_tfidf_w2v=tfidf_w2v(X_train,fasttext_word_embeddings,fasttext_words)
X_cv_fasttext_tfidf_w2v=tfidf_w2v(X_cv,fasttext_word_embeddings,fasttext_words)
X_test_fasttext_tfidf_w2v=tfidf_w2v(X_test,fasttext_word_embeddings,fasttext_words)
X_train_fasttext_tfidf_w2v=pd.DataFrame(X_train_fasttext_tfidf_w2v,columns=['embed_'+str(i) for i in range(300)])
X_cv_fasttext_tfidf_w2v=pd.DataFrame(X_cv_fasttext_tfidf_w2v,columns=['embed_'+str(i) for i in range(300)])
X_test_fasttext_tfidf_w2v=pd.DataFrame(X_test_fasttext_tfidf_w2v,columns=['embed_'+str(i) for i in range(300)])
print('\nAfter fasttext-tfidf \n')
print('Shapes of tfidf-w2v features on train data : ',X_train_fasttext_tfidf_w2v.shape)
print('Shapes of tfidf-w2v features on cv data : ',X_cv_fasttext_tfidf_w2v.shape)
print('Shapes of tfidf-w2v features on test data : ',X_test_fasttext_tfidf_w2v.shape)


After fasttext-tfidf 

Shapes of tfidf-w2v features on train data :  (6659, 300)
Shapes of tfidf-w2v features on cv data :  (940, 300)
Shapes of tfidf-w2v features on test data :  (1608, 300)


In [ ]:
X_train_glove_w2v=get_embedding_features(X_train,glove_word_embeddings,glove_words)
X_cv_glove_w2v=get_embedding_features(X_cv,glove_word_embeddings,glove_words)
X_test_glove_w2v=get_embedding_features(X_test,glove_word_embeddings,glove_words)
X_train_glove_w2v=pd.DataFrame(X_train_glove_w2v,columns=['embed_'+str(i) for i in range(300)])
X_cv_glove_w2v=pd.DataFrame(X_cv_glove_w2v,columns=['embed_'+str(i) for i in range(300)])
X_test_glove_w2v=pd.DataFrame(X_test_glove_w2v,columns=['embed_'+str(i) for i in range(300)])
print('\nAfter glove-w2v\n')
print('Shapes of tfidf-w2v features on train data : ',X_train_glove_w2v.shape)
print('Shapes of tfidf-w2v features on cv data : ',X_cv_glove_w2v.shape)
print('Shapes of tfidf-w2v features on test data : ',X_test_glove_w2v.shape)


After glove-w2v

Shapes of tfidf-w2v features on train data :  (6659, 300)
Shapes of tfidf-w2v features on cv data :  (940, 300)
Shapes of tfidf-w2v features on test data :  (1608, 300)


In [ ]:
X_train_glove_tfidf_w2v=tfidf_w2v(X_train,glove_word_embeddings,glove_words)
X_cv_glove_tfidf_w2v=tfidf_w2v(X_cv,glove_word_embeddings,glove_words)
X_test_glove_tfidf_w2v=tfidf_w2v(X_test,glove_word_embeddings,glove_words)
X_train_glove_tfidf_w2v=pd.DataFrame(X_train_glove_tfidf_w2v,columns=['embed_'+str(i) for i in range(300)])
X_cv_glove_tfidf_w2v=pd.DataFrame(X_cv_glove_tfidf_w2v,columns=['embed_'+str(i) for i in range(300)])
X_test_glove_tfidf_w2v=pd.DataFrame(X_test_glove_tfidf_w2v,columns=['embed_'+str(i) for i in range(300)])
print('\nAfter glove-tfidf_w2v \n')
print('Shapes of tfidf-w2v features on train data : ',X_train_glove_tfidf_w2v.shape)
print('Shapes of tfidf-w2v features on cv data : ',X_cv_glove_tfidf_w2v.shape)
print('Shapes of tfidf-w2v features on test data : ',X_test_glove_tfidf_w2v.shape)


After glove-tfidf_w2v 

Shapes of tfidf-w2v features on train data :  (6659, 300)
Shapes of tfidf-w2v features on cv data :  (940, 300)
Shapes of tfidf-w2v features on test data :  (1608, 300)


In [ ]:
def get_word_char_lengths(data):
  '''This function takes input dataframe and return with length of text by wordlevel and characterleve'''
  length_features=[]
  for index,row in data.iterrows():
    text=row['Description']
    length_wordlevel=len(text.split()) # Getting the number of words
    len_charlevel=len(text) # Getting the number characters including spaces
    length_features.append([length_wordlevel,len_charlevel])
  return pd.DataFrame(length_features,columns=['length_word_level','length_char_level'])

In [ ]:
X_train_length_features=get_word_char_lengths(X_train)
X_cv_length_features=get_word_char_lengths(X_cv)
X_test_length_features=get_word_char_lengths(X_test)
print('\nAfter extracting length based features\n')
print('Shapes of length based features  on train data : ',X_train_length_features.shape )
print('Shapes of length based features  on cv data : ',X_cv_length_features.shape)
print('Shapes of length based features  on test data : ',X_test_length_features.shape)


After extracting length based features

Shapes of length based features  on train data :  (6659, 2)
Shapes of length based features  on cv data :  (940, 2)
Shapes of length based features  on test data :  (1608, 2)


In [ ]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def sentiment_score(data):
  '''This function takes dataframe as input and returns sentiment scores of text data'''
  sentiments=[]
  preprocessed_descriptions = data['Description'].values
  for text in preprocessed_descriptions:
    polarities=sid.polarity_scores(text) # Getting the sentiment scores of text
    sentiments.append(list(polarities.values()))
  return pd.DataFrame(sentiments,columns=['negative','neutral','positive','compound'])

In [ ]:
X_train_sentiment_scores=sentiment_score(X_train)
X_cv_sentiment_scores=sentiment_score(X_cv)
X_test_sentiment_scores=sentiment_score(X_test)
print('\nAfter extracting sentiment features\n')
print('Shapes of sentiment features on train data : ',X_train_sentiment_scores.shape)
print('Shapes of sentiment features on cv data : ',X_cv_sentiment_scores.shape)
print('Shapes of sentiment features on test data : ',X_test_sentiment_scores.shape)


After extracting sentiment features

Shapes of sentiment features on train data :  (6659, 4)
Shapes of sentiment features on cv data :  (940, 4)
Shapes of sentiment features on test data :  (1608, 4)


In [ ]:
# Concatenating all the features that are extracted from the input
#With fastext-w2v features
X_train_final = pd.concat([X_train_fasttext_w2v,X_train_length_features,X_train_sentiment_scores],axis=1)
X_cv_final = pd.concat([X_cv_fasttext_w2v,X_cv_length_features,X_cv_sentiment_scores],axis=1)
X_test_final = pd.concat([X_test_fasttext_w2v,X_test_length_features,X_test_sentiment_scores],axis=1)

In [ ]:
print('Shape of final train data : ',X_train_final.shape,',  y_train shape : ', y_train.shape)
print('Shape of final cv data : ',X_cv_final.shape, '   ,  y_cv shape : ',y_cv.shape)
print('Shape of final test data : ',X_test_final.shape, ',  y_test shape : ',y_test.shape)

Shape of final train data :  (6659, 306) ,  y_train shape :  (6659, 3)
Shape of final cv data :  (940, 306)    ,  y_cv shape :  (940, 3)
Shape of final test data :  (1608, 306) ,  y_test shape :  (1608, 3)


In [ ]:
#with fasttext tfidf features
X_train_final_1 = pd.concat([X_train_fasttext_tfidf_w2v,X_train_length_features,X_train_sentiment_scores],axis=1)
X_cv_final_1 = pd.concat([X_cv_fasttext_tfidf_w2v,X_cv_length_features,X_cv_sentiment_scores],axis=1)
X_test_final_1 = pd.concat([X_test_fasttext_tfidf_w2v,X_test_length_features,X_test_sentiment_scores],axis=1)

In [ ]:
#with  glove features
X_train_final_2 = pd.concat([X_train_glove_w2v,X_train_length_features,X_train_sentiment_scores],axis=1)
X_cv_final_2 = pd.concat([X_cv_glove_w2v,X_cv_length_features,X_cv_sentiment_scores],axis=1)
X_test_final_2 = pd.concat([X_test_glove_w2v,X_test_length_features,X_test_sentiment_scores],axis=1)

In [ ]:
#with glove tfidf features
X_train_final_3 = pd.concat([X_train_glove_tfidf_w2v,X_train_length_features,X_train_sentiment_scores],axis=1)
X_cv_final_3 = pd.concat([X_cv_glove_tfidf_w2v,X_cv_length_features,X_cv_sentiment_scores],axis=1)
X_test_final_3 = pd.concat([X_test_glove_tfidf_w2v,X_test_length_features,X_test_sentiment_scores],axis=1)

After feature engineering we get vector representation of text which is 306-dimensions.